In [1]:
import argparse
import numpy as np
import sys
import time

import parameters
from OrbitalHessian import OrbitalHessian

import slepc4py
from petsc4py import PETSc
Pprint = PETSc.Sys.Print

slepc4py.init(sys.argv)

In [2]:
%load_ext cython

In [27]:
%%cython --annotate
#cython: wraparound=False, boundscheck=False, initializedcheck=False, cdivision=True

import numpy as np
cimport numpy as cnp

from libc.math cimport sqrt
from libc.math cimport round
    
    
cdef double high_bound_c, low_bound_c, shift_c, small_number_c, prefactor_c, k_fermi_c, k_max_c, delta_k_c
cdef cnp.int32_t ndim_c, nexc_c, nocc_c, nkpoints_c
cdef cnp.double_t[:] exc_energies_c
cdef cnp.double_t[:, :] ki_ary_c, ka_ary_c, k_occ_ary_c
cdef cnp.int32_t[:, :] indices2vir_c, exc_idx_from_states_c
    
    
cdef void to_first_brillouin_zone(cnp.double_t[:] k):
    cdef cnp.int32_t i
    for i in range(k.shape[0]):           
        if k[i] > high_bound_c:
            k[i] -= shift_c
        elif k[i] < low_bound_c:
            k[i] += shift_c        

    
cdef double two_eri_2d(cnp.double_t[:] k1, cnp.double_t[:] k3):
    cdef double denom = 0.0
    for i in range(k1.shape[0]):
        denom += (k1[i] - k3[i]) * (k1[i] - k3[i])
    denom = sqrt(denom)
    if denom < small_number_c:
        return 0.0
    else:
        return prefactor_c / denom

    
cdef cnp.int32_t b_from_indices(cnp.int32_t[:] indices):
    return indices2vir_c[indices[0], indices[1]]
    
    
def setup(cnp.ndarray[cnp.double_t, ndim=2] ki_ary not None
                 ,cnp.ndarray[cnp.double_t, ndim=2] ka_ary not None
                 ,cnp.ndarray[cnp.double_t, ndim=2] k_occ_ary not None
                 ,double k_fermi
                 ,double k_max
                 ,cnp.ndarray[cnp.double_t, ndim=1] exc_energies not None
                 ,double small_number
                 ,double prefactor
                 ,cnp.ndarray[cnp.int32_t, ndim=2] indices2vir not None
                 ,cnp.ndarray[cnp.int32_t, ndim=2] exc_idx_from_states not None
                 ,double delta_k
                 ,cnp.int32_t nkpoints):
    
    setup_c(ki_ary, ka_ary, k_occ_ary, k_fermi, k_max, exc_energies, small_number, 
            prefactor, indices2vir, exc_idx_from_states, delta_k, nkpoints)
    

    
cdef setup_c(cnp.ndarray[cnp.double_t, ndim=2] ki_ary
                 ,cnp.ndarray[cnp.double_t, ndim=2] ka_ary
                 ,cnp.ndarray[cnp.double_t, ndim=2] k_occ_ary
                 ,double k_fermi
                 ,double k_max
                 ,cnp.ndarray[cnp.double_t, ndim=1] exc_energies
                 ,double small_number
                 ,double prefactor
                 ,cnp.ndarray[cnp.int32_t, ndim=2] indices2vir
                 ,cnp.ndarray[cnp.int32_t, ndim=2] exc_idx_from_states
                 ,double delta_k
                 ,cnp.int32_t nkpoints):
    
    global k_fermi_c
    global k_max_c
    global small_number_c
    global delta_k_c
    global prefactor_c
    global high_bound_c
    global low_bound_c
    global ndim_c
    global nexc_c
    global nocc_c
    global k_occ_ary_c
    global ki_ary_c
    global ka_ary_c
    global exc_energies_c
    global indices2vir_c
    global exc_idx_from_states_c
    global nkpoints_c
    global shift_c

    k_fermi_c = k_fermi
    k_max_c = k_max
    shift_c = 2.0 * k_max
    delta_k_c = delta_k
    small_number_c = small_number
    prefactor_c = prefactor
    high_bound_c = k_max - small_number
    low_bound_c = - k_max - small_number
    ndim_c = ki_ary.shape[1]
    nexc_c = exc_energies.shape[0]
    nocc_c = k_occ_ary.shape[0]
    nkpoints_c = nkpoints
    
    ki_ary_c = ki_ary
    ka_ary_c = ka_ary
    k_occ_ary_c = k_occ_ary
    exc_energies_c = exc_energies
    indices2vir_c = indices2vir
    exc_idx_from_states_c = exc_idx_from_states
    nrows, ncols = exc_idx_from_states.shape[0], exc_idx_from_states.shape[1]
    
def from_index_c(cnp.int32_t i_row,
                       cnp.int32_t offset, 
                       cnp.ndarray[cnp.int32_t, ndim=1] indices, 
                       cnp.ndarray[cnp.double_t, ndim=1] vals):

    global nkpoints_c
    cdef cnp.double_t[:] kb = np.zeros(ndim_c)

    cdef double norm
    cdef cnp.int32_t i_col = 0 
    cdef cnp.int32_t count = 0

    cdef cnp.double_t[:] ki = np.zeros(ndim_c)
    cdef cnp.double_t[:] ka = np.zeros(ndim_c)
    cdef cnp.double_t[:] kj = np.zeros(ndim_c)

    cdef cnp.int32_t[:] b_indices = np.zeros(ndim_c, dtype=np.int32)

    cdef cnp.int32_t i
    cdef cnp.int32_t j
    cdef cnp.int32_t occ_index
    cdef cnp.int32_t b

    # generate momentum conserving pairs
    global ka_ary_c
    global k_occ_ary_c
    global ki_ary_c
    global delta_k_c
    global exc_idx_from_states_c
    global exc_energies_c
    
    for i in range(ndim_c):
        ki[i] = ki_ary_c[i_row, i]
        ka[i] = ka_ary_c[i_row, i]
    
    for occ_idx in range(nocc_c):
        for j in range(ndim_c):
            kj[j] = k_occ_ary_c[occ_idx, j]
            kb[j] = ka[j] + kj[j] - ki[j] 
            
        to_first_brillouin_zone(kb)
       
        norm = 0.0
        for j in range(kb.shape[0]):
            norm += kb[j] * kb[j]

        norm = sqrt(norm)
        
        # Only continue if actually virtual
        if norm > k_fermi_c:
            for k in range(ndim_c):
                b_indices[k] = <int>round((kb[k] + k_max_c) / delta_k_c)
            b = b_from_indices(b_indices)
            #assert(b >= 0, 'One of the values of b is negative, implying that the map from'
            #       + ' indices to b was not accessed properly.')

            # the column index is the excitation label from j -> b
            i_col = exc_idx_from_states_c[occ_idx, b]
            # diagonal is the excitation energy
            if i_col == i_row:
                vals[count] += exc_energies_c[i_row]

            vals[count] += - two_eri_2d(ki, kj)
            indices[count] = i_col
            count += 1
    vals.shape[0] = count
    indices.shape[0] = count

In [36]:
params = parameters.Parameters(n_dimensions=2
                              ,n_k_points=45
                              ,instability_type='cRHF2cUHF'
                              ,safe_eri=False
                              )
H = OrbitalHessian(params)
np.set_printoptions(precision=4, linewidth=200)


ki, ka = np.split(params.excitations._momenta, 2, axis=1)

def inverse_mapper(ary):
    """Given array of NxM entries, make a N^M MD array corresponding where MDary[i, j ...] = index of original."""
    imax = np.max(ary)
    M = ary.shape[1]
    
    if M == 1:
        inv_map = - np.ones((imax + 1), dtype=np.int32)
        for idx, i in enumerate(ary):
                inv_map[i] = idx
        return inv_map
    
    if M == 2:
        inv_map = - np.ones((imax + 1, imax + 1), dtype=np.int32)
        for idx, (i, j) in enumerate(ary):
                inv_map[i, j] = idx
        return inv_map
    
    if M == 3:
        inv_map = - np.ones((imax + 1, imax + 1, imax + 1), dtype=np.int32)
        for idx, (i, j, k) in enumerate(ary):
                inv_map[i, j, k] = idx
        return inv_map
        
vmap = inverse_mapper(params.states.virtual_indices)
emap = inverse_mapper(params.excitations.indices)

indices = np.zeros(params.excitations.n, dtype=np.int32)
vals = np.zeros(params.excitations.n, dtype=np.float)

setup(ki, ka, params.states.occupied_momenta, 
                params.k_fermi, params.k_max, 
                params.excitations.energies, 1e-10, params.eri.prefactor, vmap, emap, 
      params.k_grid_spacing, params.n_k_points)
from_index_c(0, 0, indices, vals)

indices = np.zeros(params.excitations.n, dtype=np.int32)
vals = np.zeros(params.excitations.n, dtype=np.float)

def cython_loop():  
    from_index_c(0, 0, indices, vals)
    for i, v in zip(indices, vals):
        pass
    
def python_loop():
    for i in H.A.row_generator(0, 0):
        pass

In [37]:
%timeit python_loop()

100 loops, best of 3: 4.53 ms per loop


In [39]:
%time cython_loop()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 55.6 µs
